In [2]:
import inro.modeller as m
import csv
mm = m.Modeller()
eb = m.Modeller().emmebank
scenario = eb.scenario(1)
network = scenario.get_network()
matrix_id = "mf12"

In [82]:
#convert megazone demand csv file to representative zone csv file, for EMME assigment
import numpy
sourcefile = "C:\Users\mnasterska\Documents\Projects\GGH\SketchInputs\AutoDemand.csv"
destfile = "C:\Users\mnasterska\Documents\Projects\GGH\SketchInputs\AutoDemand2.csv"
mapfile = "C:\Users\mnasterska\Documents\Projects\GGH\SketchInputs\RepresentativeZone.csv"



repmap ={}
indexmap={}
origins = eb.matrix(matrix_id).get_data().indices[0]
destinations = eb.matrix(matrix_id).get_data().indices[1]

#create index map TODO: separate for origins and destinations?
for index in range(len(origins)):
    indexmap[origins[index]]= index+1

matrix = [[0.0 for r in range(len(destinations)+1)]for c in range(len(origins)+1)]

matrix[0][0] = 'p/q/[val]'

for index in range(len(origins)):
    matrix[0][index+1] = origins[index]
for index in range(len(destinations)):
    matrix[index+1][0] = destinations[index]

#get representative zone mapping
with open(mapfile,"r") as mapf:
    lines = mapf.readlines()
    for line in lines:
        Curline = line.rstrip("\n").split(",")
        MegaZone = Curline[1]
        repmap[MegaZone] = Curline[0]

#get megazone demand
with open (sourcefile,"r") as demandf:
    header = demandf.readline()[1:].rstrip("\n").split(",")
    lines = demandf.readlines()
    for line in lines:
        Curline = line.rstrip("\n").split(",")
        origin = Curline[0]
        for index in range(len(Curline[1:])):
            matrix[indexmap[int(repmap[origin])]][indexmap[int(repmap[header[index]])]] = Curline[index+1]
  

['1', '2', '3', '4', '1401', '1402', '1501', '1502', '1503', '1601', '1602', '1801', '1802', '1803', '1804', '1805', '1806', '1807', '1808', '1809', '1810', '1811', '1812', '1813', '1814', '1815', '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908', '1909', '1910', '1911', '1912', '1913', '1914', '1915', '1916', '1917', '2002', '2003', '2004', '2005', '2006', '2007', '2101', '2102', '2103', '2104', '2106', '2107', '2108', '2201', '2202', '2301', '2302', '2303', '2304', '2305', '2401', '2402', '2403', '2404', '2405', '2406', '2407', '2408', '2409', '2501', '2502', '2503', '2504', '2601', '2602', '2603', '2604', '2605', '2606', '2607', '2801', '2901', '2902', '3001', '3002', '3003', '3004', '4301', '4302', '4303', '4304', '4305', '4306', '4307', '4308', '15021', '18011', '18081', '19021', '19061', '19081', '19121', '20011', '20021', '20031', '20061', '20071', '21031', '21061', '23021', '24021', '24041', '24051', '25021', '26071', '29021', '30021', '30031', '30032', '43061']


In [83]:

#export to csv
with open(destfile,"wb") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter = ",", quoting = csv.QUOTE_MINIMAL)
    for row in matrix:
        csvwriter.writerow(row)

#import csv matrix into emme
import_csv_matrix =  m.Modeller().tool("inro.emme.data.matrix.import_matrix_from_csv")
import_csv_matrix(matrix = eb.matrix("mf12"),
                  file_path = destfile,
                  scenario = scenario)
                  

In [4]:
#Auto Assignment - unconstrained
#create copy of network, 
copy_scenario =  m.Modeller().tool("inro.emme.data.scenario.copy_scenario")
copy_scenario(from_scenario = eb.scenario(1), 
              scenario_id = 4, 
              scenario_title = "Unconstrained Auto Assignment",
             overwrite = True)
 


Scenario(4)

In [8]:
#change VDFs to free-flow,
scenario = eb.scenario(4)
network = scenario.get_network()

for link in network.links():
    if link.volume_delay_func != 0:
        link.volume_delay_func = 90

scenario.publish_network(network)

In [9]:
#run standard assignment
spec = {
    "type": "STANDARD_TRAFFIC_ASSIGNMENT",
    "background_traffic":None,
    "path_analysis": None,
    "classes": [ 
        {
            "mode": "c",
            "demand": matrix_id,
            "generalized_cost": None,
            "results": {
                 "od_travel_times": {
                    "shortest_paths": None
                },
                 "link_volumes": None,
                 "turn_volumes": None
            },
            "analysis": None
        }
    ],
    "stopping_criteria": {
        "max_iterations": 80,
        "best_relative_gap": 0.0,
        "relative_gap": 0.01,
        "normalized_gap": 0.0
    },
    "performance_settings": {
        "number_of_processors": 5
    }
}    
auto_assign =  m.Modeller().tool("inro.emme.traffic_assignment.standard_traffic_assignment")
auto_assign(scenario = scenario, specification = spec)


Error: One or more extra function parameters are not valid